# COURSERA DATA SCIENCE PROFESSIONAL CERTIFICATE
### CAPSTONE PROJECT - The battle of Neighborhoods

## Table of Contents

1. [Problem Description](#intro)


2. [Data](#Data)

    2.1 [Introduction: Data description](#data_intro)

    2.2 [Load modules](#Load_modules)

    2.3 [Load and prepare income data by postal code](#Load_income)

    2.4 [Load and prepare geographical data](#Load_geo)

    2.5 [Merge income and geographical data](#Merge_income_geo)

    2.6 [Retreive venue data by postal code](#Venues)

## Problem Description <a class="anchor" id="intro"></a>

We will consider the problem of finding an optimal location for a new Coffee Shop in the city of Toronto.

We will need to find an appropriate location for the owner of a future new Coffee Shopp. Finding a suitable location if of paramount importance, as it is one of the most important element to ensure the economic viability of the project. This is even more important ina  big city like Toronto, where there could be a big demand, but also a lot of competition for costumers.

Upon discussion with the owner, we identified the characteristics of an interesting location. Such a location would be in a neighborhood with a **high number of potential costumers**, and with **low competition**, meaning a  number of already implanted Coffee Shops as low as possible.

Most of the work will be devoted to find a way to identify neighborhoods with a high number of potential costumers. This will be done using the locations of other venues. We will start from the assumption that already installed and successful Coffee Shops are in interesting locations, and that such locations can be characterized in terms of the kind of venues present in the vicinity. Therefore, we will start by identifying the characteristics of the Neighborhoods where successful Coffee Shops are implanted. 

Moreover, we are asked to asses whether the economic level of the neighborhood has an influence on the number of potential costumers. If this was not the case, a **rich neighborhood** would be preferred for the owner, as he whishes to open a high-end Coffee Shop, which in his opinion would be more profitable.

Based on these findings, we will create an evaluation algorithm applicable to any neighborhood, which takes into account the characteristics of the neighborhood in terms of venues present as well as potencially the average income, but penalises the presence of other Coffee Shops. Such evaluation algorithm will provide the owner with an ordered list of the most suitable areas where to open his new Coffee Shop.

## Data <a class="anchor" id="Data"></a>

### Introduction: Data description <a class="anchor" id="data_intro"></a>

We will need different geographical and socio-economic data for the city of Toronto, as well as data on the locations of installed venues.

The list of the Neighborhoods of Toronto can be found on the **Wikipedia** web page http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M . Instead of Neighborhoods, we will rather use **Postal Codes Areas** to obtain an even more refined grid. The geographical coordinates of each Neighborhood or Postal Code Area will be obtained using *Google Maps API geocoding*.

We will collect data on the average income in each neighborhood from the **official page of the Canadian Governement**, at https://open.canada.ca/en . These data will have to be cleaned and prepared, then merged with the geographical data described above.

Finally, we will get data on venues present in or near each Neighborhood using **Foursquare's databsase**.

### Load modules <a class="anchor" id="Load_modules"></a>

In [ ]:
###Temporarily suppress warnings
#import warnings
#def fxn():
#    warnings.warn("deprecated", DeprecationWarning)
#
#with warnings.catch_warnings():
#    warnings.simplefilter("ignore")
#    fxn()

In [1]:
import pandas as pd
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# use the inline backend to generate the plots within the browser
%matplotlib inline 

mpl.style.use('ggplot') # optional: for ggplot-like style

In [3]:
import json # library to handle JSON files. Will be used to retreive data through the Foursquare API
import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge beautifulsoup4 --yes   #install beautifulsoup, a library to scrap data from web pages
import bs4  #import the Beautifulsoup library

import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.3       |     pyhb0f4dca_0          86 KB  conda-forge
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    certifi-2020.11.8          |   py36h5fab9bb_0         150 KB  conda-forge
    soupsieve-2.0.1            |             py_1          30 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         410 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-for

### Load and prepare income data by postal code   <a class="anchor" id="Load_income"></a>

We will start by retriving the income data from the Canadian Governement web page. The latest data are from 2016. However, as we will only use it to compare the relative average income between Postal Areas, we can reasonably assume that in the area of interest (probably located near the center of Toronto), changes have not been so important as to radically change the picture.

Unfortunately, the full database contains fat too much information for our needs, which makes it hard to read. After downloading, loading it into a DataFrame, and analysing it, we realise that we need to perform some cleaning operations: select the column concerning the *Total income* per person (irrespectively to their family status), drop headers and two column levels, and slice the data to select only the data concerning the neighborhoods of Toronto (excluding the general data for the whole city and region).

In [4]:
# load income and postal code data, downloaded from https://open.canada.ca/en
PCdata = pd.read_excel('PCdata.xls', header=[1,2,3], skipfooter=4)
PCdata.head()

A                                       \
  General data                                        
        CityID Postal area Postal walk Level of geo   
0         9099      Z99099                       12   
1         9010      A99010                       11   
2          425      A0N1A0        XXXX            9   
3          307      A0K1A0        XXXX            9   
4           70      A0B1A0        XXXX            9   

                             Lone-parent families                          \
                                                #  Total income             
                  Place name   Unnamed: 5_level_2 # of families     $'000   
0                     CANADA              1420450       1420450  80054445   
1  NEWFOUNDLAND AND LABRADOR                25250         25250   1317520   
2                  AGUATHUNA                   20            20       740   
3               ANCHOR POINT                    X             X         X   
4               ARNOLDS COVE                   30            30      2165   

                             ...       All persons                             \
                             ... Employment income Wages/salaries/commissions   
  Median $ Provincial index  ...      Canada index               # of persons   
0    43630              ...  ...               100                   17959510   
1    39140              100  ...              93.9                     279140   
2    37410             95.6  ...              73.7                        100   
3        X                X  ...              93.5                        180   
4    51930            132.7  ...              90.1                        560   

                                                                              \
             Self-employment income           Employment Insurance             
       $'000           # of persons     $'000         # of persons     $'000   
0  851506545                2937770  50393425              2458020  17974555   
1   12913175                  27810    649455                92570    926520   
2          X                      X         X                   50       465   
3       7720                     60      4295                  150      2090   
4      26930                     50       480                  250      2615   

                                       
                                       
    EDR Provincial index Canada index  
0     2              ...          100  
1   6.8              100          340  
2  14.5            213.2          725  
3  17.4            255.9          870  
4   9.5            139.7          475  

[5 rows x 115 columns]

In [5]:
#select only the general informations and the data concerning all people irrespective of their family status
PCall = PCdata[['A','All persons']]
PCall.head()

A                                       \
  General data                                        
        CityID Postal area Postal walk Level of geo   
0         9099      Z99099                       12   
1         9010      A99010                       11   
2          425      A0N1A0        XXXX            9   
3          307      A0K1A0        XXXX            9   
4           70      A0B1A0        XXXX            9   

                                     All persons                           \
                                               # Total income               
                  Place name Unnamed: 93_level_2 # of persons       $'000   
0                     CANADA            34465690     26812650  1278307850   
1  NEWFOUNDLAND AND LABRADOR              518340       421580    19107090   
2                  AGUATHUNA                 250          220        6010   
3               ANCHOR POINT                 340          270       16325   
4               ARNOLDS COVE                 970          820       37635   

                             ...                                               \
                             ... Employment income Wages/salaries/commissions   
  Median $ Provincial index  ...      Canada index               # of persons   
0    33920              ...  ...               100                   17959510   
1    31540              100  ...              93.9                     279140   
2    20840             66.1  ...              73.7                        100   
3    35520            112.6  ...              93.5                        180   
4    32360            102.6  ...              90.1                        560   

                                                                              \
             Self-employment income           Employment Insurance             
       $'000           # of persons     $'000         # of persons     $'000   
0  851506545                2937770  50393425              2458020  17974555   
1   12913175                  27810    649455                92570    926520   
2          X                      X         X                   50       465   
3       7720                     60      4295                  150      2090   
4      26930                     50       480                  250      2615   

                                       
                                       
    EDR Provincial index Canada index  
0     2              ...          100  
1   6.8              100          340  
2  14.5            213.2          725  
3  17.4            255.9          870  
4   9.5            139.7          475  

[5 rows x 27 columns]

In [6]:
# clean first level of column's index 
PCclean = PCall.droplevel(level=0, axis=1)
PCclean.head()

General data                                       \
        CityID Postal area Postal walk Level of geo   
0         9099      Z99099                       12   
1         9010      A99010                       11   
2          425      A0N1A0        XXXX            9   
3          307      A0K1A0        XXXX            9   
4           70      A0B1A0        XXXX            9   

                                               # Total income              \
                  Place name Unnamed: 93_level_2 # of persons       $'000   
0                     CANADA            34465690     26812650  1278307850   
1  NEWFOUNDLAND AND LABRADOR              518340       421580    19107090   
2                  AGUATHUNA                 250          220        6010   
3               ANCHOR POINT                 340          270       16325   
4               ARNOLDS COVE                 970          820       37635   

                             ... Employment income Wages/salaries/commissions  \
  Median $ Provincial index  ...      Canada index               # of persons   
0    33920              ...  ...               100                   17959510   
1    31540              100  ...              93.9                     279140   
2    20840             66.1  ...              73.7                        100   
3    35520            112.6  ...              93.5                        180   
4    32360            102.6  ...              90.1                        560   

             Self-employment income           Employment Insurance            \
       $'000           # of persons     $'000         # of persons     $'000   
0  851506545                2937770  50393425              2458020  17974555   
1   12913175                  27810    649455                92570    926520   
2          X                      X         X                   50       465   
3       7720                     60      4295                  150      2090   
4      26930                     50       480                  250      2615   

                                       
    EDR Provincial index Canada index  
0     2              ...          100  
1   6.8              100          340  
2  14.5            213.2          725  
3  17.4            255.9          870  
4   9.5            139.7          475  

[5 rows x 27 columns]

In [7]:
#select only data for the city of Toronto
PCT = PCclean[PCclean.droplevel(level=0, axis=1)['Place name'] == 'TORONTO']  # select only data for the city of Toronto
PCT = PCT[PCT.droplevel(level=0, axis=1)['Level of geo'] == 3 ]   # select only data for the neighborhoods, exclude the data for the whole city
PCT.head()

General data                                                  \
           CityID Postal area Postal walk Level of geo Place name   
6160         2986      M4C                           3    TORONTO   
6161         2986      M4E                           3    TORONTO   
6162         2986      M4G                           3    TORONTO   
6163         2986      M4H                           3    TORONTO   
6164         2986      M4J                           3    TORONTO   

                       # Total income                                     ...  \
     Unnamed: 93_level_2 # of persons    $'000 Median $ Provincial index  ...   
6160               14850        11340   510205    30560             90.3  ...   
6161               22440        17190  1406825    50490            149.2  ...   
6162                 870          760    59755    45860            135.5  ...   
6163                 870          700    31765    34490            101.9  ...   
6164               16690        12720   662890    33140             97.9  ...   

     Employment income Wages/salaries/commissions           \
          Canada index               # of persons    $'000   
6160             111.9                       7390   376400   
6161             157.4                      11890  1000605   
6162             115.5                        470    29595   
6163             125.5                        440    22100   
6164             127.7                       8140   489665   

     Self-employment income         Employment Insurance             \
               # of persons   $'000         # of persons $'000  EDR   
6160                   1500   25945                  810  6315  1.6   
6161                   2810  115435                 1030  8695  0.8   
6162                    140    3820                   30   200  0.6   
6163                     80    1545                   40   280  1.2   
6164                   1840   39355                  790  6490  1.2   

                                    
     Provincial index Canada index  
6160            106.7           80  
6161             53.3           40  
6162               40           30  
6163               80           60  
6164               80           60  

[5 rows x 27 columns]

In [8]:
#selects only the median income column
PCTI = PCT.loc(axis=1)[pd.IndexSlice[:,['Postal area','Median $']]].copy()
PCTI.drop('Employment income', axis=1, level=0, inplace=True)
PCTI.head()

,General data,Total income
,Postal area,Median $
6160,M4C,30560
6161,M4E,50490
6162,M4G,45860
6163,M4H,34490
6164,M4J,33140


In [9]:
#simplify the headings and columns names
PCTI = PCTI.droplevel(level=0, axis=1)
PCTI.rename(columns={'Postal area':'Postal Code', 'Median $':'Median income'}, inplace=True)
PCTI.head()

,Postal Code,Median income
6160,M4C,30560
6161,M4E,50490
6162,M4G,45860
6163,M4H,34490
6164,M4J,33140


In [10]:
PCTI.shape

(43, 2)

We have almost finished with the socio-economic data, we just have to fix one last small problem: the Postal Codes in the Data Frame have a few spaces after the actual code. Let's look for example at the first value:

In [11]:
PCTI.iloc[0,0]

'M4C   '

We have to remove the spaces in order to be able to later join this data frame with the one we will create containing geographical and venues data.

In [12]:
PCTI['Postal Code'] = PCTI['Postal Code'].str.strip()
PCTI.iloc[1,0]

'M4E'

### Load and prepare geographical data <a class="anchor" id="Load_geo"></a>

#### Scrape the data

I will use the Python method *read_html* together with the *BeautifulSoup* librery to retreive the list of Postal Areas and Neighborhood names for the city of Toronto from the Wikipedia web page.

In [13]:
# create the list of DataFrames from all of the tables on the web page -  We'll only need the first dataframe
FSA = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='bs4')

In [14]:
# keep only the first DataFrame, which is the one we need
FSA = FSA[0]
FSA

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Prepare the data

We rename the columns in a more meaningful way and clean the DataFrame dropping the rows with '*Not assigned*' Borough.

In [15]:
#change the name of the columns
FSA.columns=['Postal Code', 'Borough', 'Neighborhood']
FSA =FSA[FSA.Borough != 'Not assigned']  #drops the rows with 'Not assigned' Borough
FSA

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [16]:
PostalCodes = FSA['Postal Code'].nunique()  #counts the number of different Postal Codes.
DF_rows = FSA.shape[0]

print('There are {} different Postal Codes and {} rows in the DataFrame.'.format(PostalCodes, DF_rows))

There are 103 different Postal Codes and 103 rows in the DataFrame.


**Data integrity check**: we can see that we have here 103 different Postal Codes, and 103 rows in the DataFrame, which means that no Postal Code is listed twice. We can also check that there are no 'Not assigned' Neighborhoods left in the DataFrame.

In [17]:
FSA[FSA['Neighborhood'] == 'Not assigned']  #checks if there are still 'Not assigned' Neighborhoods 

,Postal Code,Borough,Neighborhood


#### Retreiving geographical coordinates

As the Google Maps geocoding API is not always very stable, we will load a pre-compiled copy of the geographical coordinates of each Postal Area.

In [18]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge income and geographical data <a class="anchor" id="Merge_income_geo"></a>

We will first join the two DataFrames of geographical data to append the coordinates of each Postal Code area to the name of the Borough and its Neighborhoods.

To be sure that all Postal Codes are taken into account, we use the 'outer' join, and check if the dimension of the resulting DataFrame has grown (which would mean that there were rows in one of the DataFrames that were not presents in the other one).

In [19]:
result = pd.merge(FSA, coordinates, how='outer', on=['Postal Code', 'Postal Code'])
result

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


As above, we have 103 Postal Code Areas. 

We will now proceed in a similar way to merge the geographical data with the income data.

In [20]:
GItoronto = pd.merge(PCTI, result, how='left', on=['Postal Code', 'Postal Code'])
GItoronto.head()

,Postal Code,Median income,Borough,Neighborhood,Latitude,Longitude
0,M4C,30560,East York,Woodbine Heights,43.695344,-79.318389
1,M4E,50490,East Toronto,The Beaches,43.676357,-79.293031
2,M4G,45860,East York,Leaside,43.709060,-79.363452
3,M4H,34490,East York,Thorncliffe Park,43.705369,-79.349372
4,M4J,33140,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106


### Retreive venue data by postal code   <a class="anchor" id="Venues"></a> 

Using Foursuqare's API we will retreive the most popular venues in every Neighborhood. First, we prepare the credentials for the API calls, then we  define a function that retrieves the venues located within 500m of the center of each given Neighborhood, and finally apply it to all of the Postal Code Areas in Toronto.

In [21]:
# prepare the credentials for API calls
CLIENT_ID = #removed for privacy
CLIENT_SECRET = #removed for privacy
VERSION = '20201112'  #Foursquare version
LIMIT = 100 
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [22]:
# get venues near a given list of neighborhoods. 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lon in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            radius, 
            LIMIT)
        
    
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        #print(type(results))
        #print(results)
    
    
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lon, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            #v['venues']['categories'].apply(get_category_type, axis=1) )for v in results])
            v['venue']['categories'][0]['name']) for v in results])
                

    nearby_venues = pd.DataFrame([item for venue_data in venues_list for item in venue_data])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# apply the function to the different Boroughs in Toronto.

Toronto_venues = getNearbyVenues(names=GItoronto['Postal Code'],
                                   latitudes=GItoronto['Latitude'],
                                   longitudes=GItoronto['Longitude']
                                  )
Toronto_venues.shape

M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6N
M6P
M6R
M6S
M9M


(1288, 7)

We can now show the DataFrame with the data obtained onthe different venues.

In [31]:
# Show the DataFrame with the venues
Toronto_venues

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4C,43.695344,-79.318389,East York Memorial Arena,43.697224,-79.315397,Skating Rink
1,M4C,43.695344,-79.318389,East York Curling Club,43.696827,-79.313658,Curling Ice
2,M4C,43.695344,-79.318389,The Beer Store,43.693731,-79.316759,Beer Store
3,M4C,43.695344,-79.318389,Stan Wadlow Park,43.697836,-79.314303,Park
4,M4C,43.695344,-79.318389,Woodbine & Cosburn,43.696456,-79.316614,Intersection
...,...,...,...,...,...,...,...
1283,M6S,43.651571,-79.484450,West End Mamas,43.648703,-79.484919,Health Food Store
1284,M6S,43.651571,-79.484450,(The New) Moksha Yoga Bloor West,43.648658,-79.485242,Yoga Studio
1285,M6S,43.651571,-79.484450,The Coffee Bouquets,43.648785,-79.485940,Coffee Shop
1286,M6S,43.651571,-79.484450,Think Fitness,43.647966,-79.486462,Gym


In [25]:
# Counts unique categories
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


In [30]:
#Save DataFrame as .csv file
#Toronto_venues.to_csv('Toronto_venues1112.csv')

In [ ]:
#read data from .csv file
#Toronto_venues = pd.read_csv('Toronto_venues1112.csv')
#Toronto_venues.drop('Unnamed: 0', axis=1, inplace=True)
#Toronto_venues